# Learning Resources
Learning materials from Youtube channel: AI Jason - https://www.youtube.com/watch?v=69bH4IHZivs

## Some useful links for further improve your prompting
### FlowGPT 
See references and gain ideas from others prompts
- https://flowgpt.com/

### GPT Prompt Engineer 
Use gpt to generate prompt for your task and evaluate each generated prompt 
- https://github.com/mshumer/gpt-prompt-engineer

### PromptsRoyale  
Created based on GPT Prompt Engineer (Can used for evaluating your own prompts by changing the generated prompts)
- https://www.promptsroyale.com/

In [ ]:
# !pip install guidance
# !pip install openai

In [3]:
import guidance

guidance.llm = guidance.llms.OpenAI("text-davinci-003")

### Guardrail the LLM output structure

In [73]:
program = guidance('''
once {{people}} said:
"Life would be {{gen "story"}}"
Counter argument: "{{gen "argument"}}"
''')

quote = program(people="Stephen William Hawking")

quote

Stop program 
once Stephen William Hawking said:
"Life would be tragic if it weren't funny." "
Counter argument: " Life can still be meaningful and fulfilling even without humor." "

### Restrict the LLM answer options

In [64]:
options = ['Yes','No','Maybe']

program = guidance('''
Is this following sentence aggressive?
Sentence: {{example}}
Answer: {{select "answer" options=options}}
''')

quote1 = program(example="Your services are excellent!", options=options)

quote1

Stop program 
Is this following sentence aggressive?
Sentence: Your services are excellent! 
Answer: No

### If condition & hidden generation

In [65]:
options = ['Yes','No']

program = guidance('''
User: {{query}}

{{#block hidden=True~}}
Is the user query rude or offensive?
Answer: {{select "rude" options=options}}
{{~/block~}}

{{#if rude=="Yes"}} Assistant: Please be polite{{~/if~}}
{{#if rude=="No"}} Assistant: {{gen "answer"}}{{~/if~}}

''')

quote2 = program(query="Your service is execellent!", options=options)

quote2

Stop program 
User: Your service is execellent! 

 Assistant: Thank you! I'm glad you're happy with our service. Is there anything else I can help you with?

### Generate email with if condition 

In [35]:
priority_score = ['low','medium','high']

email = guidance('''
{{#block hidden=True~}}
Here is the customer message we received: {{email}}
Please give it a priority score.
priority score: {{select "priority" options=priority_score}}
{{~/block~}}

{{#block hidden=True~}}
You are a world class customer support; Your goal is to generate an response based on the customer message;
Here is the customer message to respond: {{email}}
Generating an opening & one paragraph of response to the customer message based on the priority score: {{priority}}
{{gen "email_response"}}
{{~/block~}}

{{email_response}}
{{#if priority=="high"}} Would have to setup a call this week, here is the calendar link https://sample.com {{/if}}
Best regards
Qiming

''')

quote3 = email(email="Hi, I am very satisfy with your service and was intended to have some collaboration with you and your team. Please contact me if you are interested.", priority_score=priority_score)

Stop program 
 
Thank you for your kind words! We are always looking for ways to collaborate with our customers and are delighted to hear that you are interested in working with us. We would love to discuss the possibilities further and learn more about how we can work together. Please feel free to reach out to us with any questions or ideas you may have. We look forward to hearing from you! 
 
Best regards
Qiming

### Generate Chart

In [48]:
import urllib.parse

def generate_chart(query):
    
    def parse_chart_link(chart_details):
        encoded_chart_details = urllib.parse.quote(chart_details, safe='')

        output = "![](https://quickchart.io/chart?c=" + encoded_chart_details + ")"

        return output
    
    examples = [
        {
            'input': "Make a chart of the 5 tallest mountains",
            'output': {"type":"bar","data":{"labels":["Mount Everest","K2","Kangchenjunga","Lhotse","Makalu"], "datasets":[{"label":"Height (m)","data":[8848,8611,8586,8516,8485]}]}}
        },
        {
            'input': "Create a pie chart showing the population of the world by continent",
            'output': {"type":"pie","data":{"labels":["Africa","Asia","Europe","North America","South America","Oceania"], "datasets":[{"label":"Population (millions)","data": [1235.5,4436.6,738.8,571.4,422.5,41.3]}]}}
        }
    ]

    guidance.llm = guidance.llms.OpenAI("text-davinci-003") 

    chart = guidance(
    '''    
    {{#block hidden=True~}}
    You are a world class data analyst, You will generate chart output based on a natural language;

    {{~#each examples}}
    Q:{{this.input}}
    A:{{this.output}}
    ---
    {{~/each}}
    Q:{{query}}
    A:{{gen 'chart' temperature=0 max_tokens=500}}    
    {{/block~}}
    
    Hello here is the chart you want
    {{parse_chart_link chart}}
    ''')

    return chart(query=query, examples=examples, parse_chart_link=parse_chart_link)

In [52]:
chart = generate_chart("Create a pie chart showing the popularity of AI content types")

chart

Stop program 
 Hello here is the chart you want
 ![](https://quickchart.io/chart?c=%20%7B%27type%27%3A%20%27pie%27%2C%20%27data%27%3A%20%7B%27labels%27%3A%20%5B%27Natural%20Language%20Processing%27%2C%20%27Computer%20Vision%27%2C%20%27Robotics%27%2C%20%27Machine%20Learning%27%2C%20%27Deep%20Learning%27%5D%2C%20%27datasets%27%3A%20%5B%7B%27label%27%3A%20%27Popularity%20%28%25%29%27%2C%20%27data%27%3A%20%5B20%2C%2015%2C%2010%2C%2025%2C%2030%5D%7D%5D%7D%7D)

 ![](https://quickchart.io/chart?c=%20%7B%27type%27%3A%20%27pie%27%2C%20%27data%27%3A%20%7B%27labels%27%3A%20%5B%27Natural%20Language%20Processing%27%2C%20%27Computer%20Vision%27%2C%20%27Robotics%27%2C%20%27Machine%20Learning%27%2C%20%27Deep%20Learning%27%5D%2C%20%27datasets%27%3A%20%5B%7B%27label%27%3A%20%27Popularity%20%28%25%29%27%2C%20%27data%27%3A%20%5B20%2C%2015%2C%2010%2C%2025%2C%2030%5D%7D%5D%7D%7D)

### Embed text to image

In [31]:
def generate_story(story_idea):
        
    story = guidance('''
{{#block hidden=True~}}
You are a world class story teller; Your goal is to generate a short tiny story less than 200 words based on a story idea;

Here is the story idea: {{story_idea}}
Story: {{gen 'story' temperature=0}}
{{/block~}}

{{#block hidden=True~}}
You are a world class AI artiest who are great at generating text to image prompts for the story; 
Your goal is to generate a good text to image prompt and put it in a url that can generate image from the prompt;

Story: You find yourself standing on the deck of a pirate ship in the middle of the ocean. You are checking if there are still people on the ship
Image url markdown: ![Image](https://image.pollinations.ai/prompt/a%203d%20render%20of%20a%20man%20standing%20on%20the%20deck%20of%20a%20pirate%20ship%20in%20the%20middle%20of%20the%20ocean)
                
Story: {{story}}
Image url markdown: {{gen 'image_url' temperature=0 max_tokens=500}})
{{~/block~}}
                     
Story: {{~story~}}
{{image_url}}
''')

    story = story(story_idea=story_idea)
    print(story)
    return story

In [33]:
generate_story("chasing backpack")

Stop program 
Story: 

The sun was setting on a warm summer evening, and the streets were bustling with people. As the sky turned a deep orange, a young boy was running down the street, a backpack clutched tightly in his hands. He was being chased by a group of men, their faces hidden in the shadows of the night.

The boy ran as fast as he could, weaving in and out of the crowds, but the men were gaining on him. He could hear their footsteps getting closer and closer, and he knew he had to find a way to escape. He ducked into an alleyway and hid behind a dumpster, his heart pounding in his chest.

The men stopped at the entrance of the alley, searching for the boy. He held his breath, hoping they wouldn't find him. After what felt like an eternity, the men finally gave up and left.

The boy let out a sigh of relief and opened his backpack. Inside was a small, golden statue, and he knew he had to get it to its rightful owner. He smiled to himself, knowing he had done the right thing, and ran off into the night. 
![Image](https://image.pollinations.ai/prompt/a%203d%20render%20of%20a%20young%20boy%20running%20down%20the%20street%20at%20night%20with%20a%20backpack%20clutched%20tightly%20in%20his%20hands%20being%20chased%20by%20a%20group%20of%20men)